In [7]:
import cv2
import time
import numpy as np
import HandTrackingModule as htm
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

################################
wCam, hCam = 640, 480
################################

cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0

detector = htm.handDetector(detectionCon=0.7)

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
# volume.GetMute()
# volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()
minVol = volRange[0]
maxVol = volRange[1]
vol = 0
volBar = 400
volPer = 0
while True:
    success, img = cap.read()
    img = cv2.flip(img, -1)    # flip the image/video 180 deg
    img = detector.findHands(img)
    lmList = detector.findPosition(img, draw=False)
    if len(lmList) != 0:
        # print(lmList[4], lmList[8])

        x1, y1 = lmList[4][1], lmList[4][2]
        x2, y2 = lmList[8][1], lmList[8][2]
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2

        cv2.circle(img, (x1, y1), 15, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, (x2, y2), 15, (255, 0, 255), cv2.FILLED)
        cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
        cv2.circle(img, (cx, cy), 15, (255, 0, 255), cv2.FILLED)

        length = math.hypot(x2 - x1, y2 - y1)
        # print(length)

        # Hand range 50 - 300
        # Volume Range -65 - 0

        vol = np.interp(length, [50, 300], [minVol, maxVol])
        volBar = np.interp(length, [50, 300], [400, 150])
        volPer = np.interp(length, [50, 300], [0, 100])
        print(int(length), vol)
        volume.SetMasterVolumeLevel(vol, None)

        if length < 50:
            cv2.circle(img, (cx, cy), 15, (0, 255, 0), cv2.FILLED)

    cv2.rectangle(img, (50, 150), (85, 400), (255, 0, 0), 3)
    cv2.rectangle(img, (50, int(volBar)), (85, 400), (255, 0, 0), cv2.FILLED)
    cv2.putText(img, f'{int(volPer)} %', (40, 450), cv2.FONT_HERSHEY_COMPLEX,
                1, (255, 0, 0), 3)


    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (40, 50), cv2.FONT_HERSHEY_COMPLEX,
                1, (255, 0, 0), 3)

    cv2.imshow("Img", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # press q to exit the video
        break


cap.release()
cv2.destroyAllWindows()

98 -154.6127575907396
121 -137.11723914891886
124 -134.86808516521822
129 -130.79658801024937
129 -131.03967536284918
129 -130.66343819842194
132 -128.3137606922461
131 -129.370774921313
129 -130.9121286588159
129 -131.13767119395195
125 -133.90667554695818
123 -135.8018513500396
122 -136.21113237754687
125 -133.8486234173743
129 -131.32502330053262
133 -128.14346049273914
133 -127.4965973740421
135 -126.24039815744302
135 -126.43594467317081
134 -126.84807911004258
136 -125.54620006885779
135 -126.01978670265136
137 -124.95720792771085
139 -123.01997682995223
139 -123.01997682995223
142 -120.66196371357833
143 -119.8373372245404
143 -120.27565544349423
144 -119.20961883328216
149 -115.32038289948999
147 -117.34998260946618
147 -117.46221181553094
137 -124.75602370224792
136 -125.51807782081796
141 -121.54700584733557
137 -124.89289904466145
143 -119.95476405023167
133 -128.253112822759
121 -136.9782373962041
113 -143.4496461651823
82 -166.98998337801825
69 -177.08066316991554
61 -183.